In [ ]:
!pip install ./diffusers

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
from importlib.metadata import version
print("diffusers version:", version("diffusers"))
print("torch version:", version("torch"))
print("transformers version:", version("transformers"))

diffusers version: 0.36.0.dev0
torch version: 2.8.0
transformers version: 4.55.2


In [9]:
# 在 notebook 单元格中运行
!pip install ipywidgets tqdm accelerate

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 15.5 MB/s  0:00:00

   -------------------- ------------------- 2/4 [ipywidgets]
   -------------------- ------------------- 2/4 [ipywidgets]
   -------------------- ------------------- 2/4 [ipywidgets]
   ------------------------------ --------- 3/4 [accelerate]
   ------------------------------ --------- 3/4 [accelerate]
   ------------------------------ --------- 3/4 [accelerate]
   ------------------------------ --------- 3/4 [accelerate]
   ------------------------------ --------- 3/4 [accelerate]
   ------------------------------ --------- 3/4 [accelerate]
   ------------------------------ --------- 3/4 [accelerate]
   ---------------------------------------- 4/4 [accelerate]



In [2]:
import torch

# Load the pipeline
if torch.cuda.is_available():
    torch_dtype = torch.bfloat16
    device = "cuda"
else:
    torch_dtype = torch.float32
    device = "cpu"
print(f"使用的是{device}")

使用的是cpu


In [3]:
from diffusers import QwenImagePipeline

model_name = "./models/Qwen-Image"
pipe = QwenImagePipeline.from_pretrained(model_name, torch_dtype=torch_dtype)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

The config attributes {'pooled_projection_dim': 768} were passed to QwenImageTransformer2DModel, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# from diffusers import DiffusionPipeline


# model_name = "./models/Qwen-Image"


# pipe = DiffusionPipeline.from_pretrained(model_name, torch_dtype=torch_dtype)
# print(f"管道已加载成功{pipe}")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


AttributeError: 'super' object has no attribute '__getattr__'

In [4]:
pipe = pipe.to(device)

positive_magic = {
    "en": ", Ultra HD, 4K, cinematic composition.", # for english prompt
    "zh": ", 超清，4K，电影级构图." # for chinese prompt
}

# Generate image
prompt = '''A coffee shop entrance features a chalkboard sign reading "Qwen Coffee 😊 $2 per cup," with a neon light beside it displaying "通义千问". Next to it hangs a poster showing a beautiful Chinese woman, and beneath the poster is written "π≈3.1415926-53589793-23846264-33832795-02384197".'''

negative_prompt = " " # Recommended if you don't use a negative prompt.


# Generate with different aspect ratios
aspect_ratios = {
    "1:1": (1328, 1328),
    "16:9": (1664, 928),
    "9:16": (928, 1664),
    "4:3": (1472, 1104),
    "3:4": (1104, 1472),
    "3:2": (1584, 1056),
    "2:3": (1056, 1584),
}

width, height = aspect_ratios["16:9"]

image = pipe(
    prompt=prompt + positive_magic["en"],
    negative_prompt=negative_prompt,
    width=width,
    height=height,
    num_inference_steps=50,
    true_cfg_scale=4.0,
    generator=torch.Generator(device="cuda").manual_seed(42)
).images[0]

image.save("example.png")

RuntimeError: Cannot get CUDA generator without ATen_cuda library. PyTorch splits its backend into two shared libraries: a CPU library and a CUDA library; this error has occurred because you are trying to use some CUDA functionality, but the CUDA library has not been loaded by the dynamic linker for some reason.  The CUDA library MUST be loaded, EVEN IF you don't directly use any symbols from the CUDA library! One common culprit is a lack of -INCLUDE:?warp_size@cuda@at@@YAHXZ in your link arguments; many dynamic linkers will delete dynamic library dependencies if you don't depend on any of their symbols.  You can check if this has occurred by using link on your binary to see if there is a dependency on *_cuda.dll library.